In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import json
import os 

root = "."

pathogen_code = "mtuberculosis"

OUTPUT = os.path.join(root, "..", "output")

In [2]:
# Define path to parameters
PATH_TO_PARAMETERS = os.path.join(OUTPUT, pathogen_code, "parameters")

# Load assays info
ASSAYS_CLEANED = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_cleaned.csv" ))
ASSAYS_CLUSTERS = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_clusters.csv" ))
ASSAYS_DATA_RANGES = pd.read_csv(os.path.join(OUTPUT, pathogen_code, "assays_data_ranges.csv" ))

# Shared columns
KEYS = ["assay_id", "activity_type", "unit"]

# Columns to take from each table
COLUMNS_CLEANED = ["assay_id", "assay_type", "assay_organism", "doc_chembl_id", "target_type", "target_chembl_id", "target_organism", "activity_type", 
                   "unit", "canonical_unit", "activities", "nan_values", "cpds", "direction", "activity_comment_counts", "standard_text_count"]
COLUMNS_CLUSTERS = ['clusters_0.3', 'clusters_0.6', 'clusters_0.85']
COLUMNS_DATA_RANGES = ["equal", 'higher', 'lower', "min_", "p1", "p25", "p50", "p75", "p99", "max_", "expert_cutoff", "dataset_type", "positives_quant", "ratio_quant", "positives_qual", "negatives_qual"]


In [3]:
def assert_unique_on_keys(df, keys, name):
    dups = df.duplicated(keys, keep=False)
    if dups.any():
        examples = df.loc[dups, keys].drop_duplicates().head(10)
        raise ValueError(
            f"{name} is NOT unique on keys {keys} (found {dups.sum()} duplicate rows).\n"
            f"Example duplicate keys (first 10):\n{examples.to_string(index=False)}")
    
assert_unique_on_keys(ASSAYS_CLEANED, KEYS, "ASSAYS_CLEANED")
assert_unique_on_keys(ASSAYS_CLUSTERS, KEYS, "ASSAYS_CLUSTERS")
assert_unique_on_keys(ASSAYS_DATA_RANGES, KEYS, "ASSAYS_DATA_RANGES")

In [4]:
ORGNISM_CURATED, TARGET_TYPE_CURATED, STRAIN, ATCC_ID, MUTATIONS, KDR, MEDIA = [], [], [], [], [], [], []

# Iterating over assays
for assay_id, activity_type, unit in ASSAYS_CLEANED[['assay_id', 'activity_type', 'unit']].values:

    # Prepare filename
    filename = "_".join([str(assay_id), str(activity_type), str(unit), 'parameters']) + ".json"
    
    # Read JSON file
    with open(os.path.join(PATH_TO_PARAMETERS, filename), "r") as file:
        par = json.load(file)

    # Store results
    ORGNISM_CURATED.append(par['organism'])
    TARGET_TYPE_CURATED.append(par['target_type'])
    STRAIN.append(par['strain'])
    # ATCC_ID.append(par['atcc_id'])
    MUTATIONS.append(";".join(par['mutations']))
    KDR.append(";".join(par['known_drug_resistances']))
    MEDIA.append(par['media'])

# Complete table
ASSAYS_CLEANED['organism_curated'] = ORGNISM_CURATED
ASSAYS_CLEANED['target_type_curated'] = TARGET_TYPE_CURATED
ASSAYS_CLEANED['strain'] = STRAIN
# assays_cleaned['atcc_id'] = ATCC_ID
ASSAYS_CLEANED['mutations'] = MUTATIONS
ASSAYS_CLEANED['known_drug_resistances'] = KDR
ASSAYS_CLEANED['media'] = MEDIA

In [5]:
ASSAYS_CLEANED = ASSAYS_CLEANED.merge(ASSAYS_CLUSTERS[KEYS + COLUMNS_CLUSTERS],on=KEYS, how="left", validate="1:1")
ASSAYS_CLEANED = ASSAYS_CLEANED.merge(ASSAYS_DATA_RANGES[KEYS + COLUMNS_DATA_RANGES],on=KEYS, how="left", validate="1:1")

In [6]:
ASSAYS_CLEANED

,assay_id,assay_type,assay_organism,doc_chembl_id,target_type,target_chembl_id,target_organism,activity_type,unit,canonical_unit,activities,nan_values,cpds,direction,activity_comment_counts,standard_text_count,organism_curated,target_type_curated,strain,mutations,known_drug_resistances,media,clusters_0.3,clusters_0.6,clusters_0.85,equal,higher,lower,min_,p1,p25,p50,p75,p99,max_,expert_cutoff,dataset_type,positives_quant,ratio_quant,positives_qual,negatives_qual
0,CHEMBL4649948,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,PERCENTEFFECT,%,%,93555,0,86589,1.0,0,0,Mycobacterium tuberculosis,ORGANISM,,,,"DPPC, cholesterol, tyloxapol based media",25146,72439,85917,93555.0,0.0,0.0,-1122.89,-39.791,-10.300,-1.066,7.879,58.950,120.27,50.0,quantitative,1268.0,0.01464,0.0,0.0
1,CHEMBL4649949,F,Mycobacterium tuberculosis,CHEMBL3988442,UNCHECKED,CHEMBL612545,NaN,PERCENTEFFECT,%,%,101515,0,86575,1.0,0,0,Mycobacterium tuberculosis,ORGANISM,,,,"7H9, glucose tyloxapol based media",25056,72123,85853,101515.0,0.0,0.0,-1111.40,-46.433,-6.945,3.342,14.630,66.173,133.09,50.0,quantitative,2181.0,0.02519,0.0,0.0
2,CHEMBL4649971,F,Mycobacterium tuberculosis,CHEMBL3988442,ORGANISM,CHEMBL360,Mycobacterium tuberculosis,PERCENTEFFECT,%,%,68619,0,68613,1.0,0,0,Mycobacterium tuberculosis,ORGANISM,,,,,18335,54118,67897,68619.0,0.0,0.0,-303.60,-47.290,-13.540,-2.846,6.803,62.022,176.21,50.0,quantitative,934.0,0.01361,0.0,0.0
3,CHEMBL4649972,F,Mycobacterium tuberculosis,CHEMBL3988442,PROTEIN COMPLEX,CHEMBL4662931,Mycobacterium tuberculosis (strain ATCC 25618 ...,PERCENTEFFECT,%,%,68616,0,68610,1.0,0,0,Mycobacterium tuberculosis,PROTEIN COMPLEX,H37Rv,,,,18638,54529,67780,68616.0,0.0,0.0,-4329.36,-46.070,-9.429,-3.115,2.931,20.768,97.51,NaN,quantitative,NaN,NaN,0.0,0.0
4,CHEMBL4649941,F,Mycobacterium tuberculosis,CHEMBL3988442,SINGLE PROTEIN,CHEMBL4662928,Mycobacterium tuberculosis (strain ATCC 25618 ...,PERCENTEFFECT,%,%,67381,0,66941,1.0,0,0,Mycobacterium tuberculosis,SINGLE PROTEIN,H37Rv,,,,17278,53010,66186,67381.0,0.0,0.0,-254.55,-28.112,-1.211,1.844,4.841,17.390,101.82,50.0,quantitative,40.0,0.00060,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10516,CHEMBL4153753,B,Mycobacterium tuberculosis H37Rv,CHEMBL4152223,SINGLE PROTEIN,CHEMBL1849,Mycobacterium tuberculosis,INHIBITION,%,%,1,1,1,1.0,1,0,Mycobacterium tuberculosis,SINGLE PROTEIN,H37Rv,,,,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10517,CHEMBL4153752,B,Mycobacterium tuberculosis H37Rv,CHEMBL4152223,SINGLE PROTEIN,CHEMBL1849,Mycobacterium tuberculosis,INHIBITION,%,%,1,1,1,1.0,1,0,Mycobacterium tuberculosis,SINGLE PROTEIN,H37Rv,,,,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10518,CHEMBL5226835,F,Mycobacterium tuberculosis variant bovis,CHEMBL5226287,ORGANISM,CHEMBL613086,Mycobacterium tuberculosis variant bovis,MIC50,umol.L-1,umol.L-1,1,0,1,-1.0,0,0,Mycobacterium tuberculosis variant bovis,ORGANISM,BCG,,,,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10519,CHEMBL5226836,B,Mycobacterium tuberculosis,CHEMBL5226287,UNCHECKED,CHEMBL612545,NaN,IC50,umol.L-1,umol.L-1,1,0,1,-1.0,0,0,Mycobacterium tuberculosis,SINGLE PROTEIN,,,,,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
COLUMNS = ["assay_id", 
           "assay_type", 
           "assay_organism", 
           "doc_chembl_id",
           "target_type", 
           "target_chembl_id",
           "target_type_curated",
           "target_organism",
           "strain",
           "mutations",  # Residue, number, residue!
           "known_drug_resistances",
           "media", 
           "activity_type", 
           "unit",
           "canonical_unit", 
           "activities", 
           "nan_values", 
           "cpds",	
           "direction", 
           "activity_comment_counts", 
           "standard_text_count",
           "clusters_0.3",
           "clusters_0.6",
           "clusters_0.85",]

In [ ]:
ASSAYS_CLEANED